
# DECISION TREE FROM SCRATCH [CLASSIFICATION USING ENTROPY]



In [ ]:
#import the necessary libraries 
import pandas as pd 
import numpy as np


In [ ]:
data=pd.read_csv("zoo.csv")
data.columns

Index(['animal_name', 'hair', 'feathers', 'eggs', 'milk', 'airborne',
       'aquatic', 'predator', 'toothed', 'backbone', 'breathes', 'venomous',
       'fins', 'legs', 'tail', 'domestic', 'catsize', 'class_type'],
      dtype='object')

In [ ]:
features=['hair', 'feathers', 'eggs', 'milk', 'airborne',
       'aquatic', 'predator', 'toothed', 'backbone', 'breathes', 'venomous',
       'fins', 'legs', 'tail', 'domestic', 'catsize']
target="class_type"

In [ ]:
data.drop(columns="animal_name",axis=1,inplace=True)

##ENTROPY


$ entropy(x) = -\sum_{for \ vals \ \in column}(P(x=vals)*log_2(P(x=vals)))$

In [ ]:
def calculate_entropy(column):
  """
  Parameters: 
      column : the column whose entropy has to be found out.
  About the   Function:
      Using entropy formula the entropy of the column is calculated
  Return Type:
      Floating point no.
  Called In:
      information_gain()
  """
  entropy =0.0
  vals,count = np.unique(column,return_counts=True) 
  # vals: the different values in the column
  # count : the frequency of the different values in that column
  
  for i in range(len(vals)):
    #the entropy formula is applied all the possible vals of the column and ultimately theies individual entropies will be added
    p=count[i]/np.sum(count)
    entropy=entropy+(-p*(np.log2(p)) )

  return entropy


##INFORMATION GAIN

In [ ]:
def information_gain(data, split_decision_feature, target_col="target"):
  """
  Parameters:
      data          = the data_frame on which the information gain has to be calculated.
      split_feature = the feature based on which the decision is to be made.
      target _col   =  required to find the weighted entropy of the split_feature 
  About the Function 
      It takes in the parameter and finds out the information gained  
      *formala* [  information_gain = total_entropy  -   weighted_entropy    ]
      total_entropy = entropy of the data_frame 
  Return type:
       information_gain float type
  Called in:
        ID3()
  """
  weighted_entropy = 0.0

  #Calculate total_entropy
  total_entropy = calculate_entropy( data[target_col] )

  #Calculate value and  frequency of each value in the split_feature column
  vals,counts  = np.unique( data[split_decision_feature] , return_counts=True )

  #based each different vals calculation of weighted entropy
  for i in range(len(vals)):

    #weight: It is the probability that each vals of the split_decision_feature occurs. 
    weight = counts[i]/( np.sum(counts) )
   
    ##################################################################################################################################################################################################################
    #  p_w : the entropy of each vals of the split decision feature.                                                                                                                                                 #
    #  function_description:                                                                                                                                                                                         #
    #     ->Here we are calling entropy function to calculated p_w.                                                                                                                                                  #
    #     ->data.where function is used to return a dataframe which tells us where in the dataset which gives us the idea of places where the particular value in vals exist                                         #
    #         suppose there is a column in a data that we are considering is 'Weather' which has vals of 'sunny','rainy' and 'cold'                                                                                  #
    #         so what data.where(data.weather==vals["sunny"]) would do is it'll return a datset which tells us where in the dataset the value of the weather column is sunny.                                        #
    #         suppose  the target_feature was "go out an play"  which has vals of "yes,"no"                                                                                                                          #
    #         np.where(data.weather==vals["sunny"]).dropna() would drop all the instances in the dataset which has null value                                                                                        #
    #         np.where(data.weather==vals["sunny"]).dropna().target_feature this function would return the instances in the target_feature column where corresponding to the weather feature the vals vwas sunny     #
    #         so this function returns the entropy of  the above case                                                                                                                                                #
    ##################################################################################################################################################################################################################     
    p_w = calculate_entropy(data.where(data[split_decision_feature] == vals[i]).dropna()[target_col])

    #calculate the weighted_entropy
    weighted_entropy = weighted_entropy+ p_w*weight
    
  #Calculate information gain
  info_gain =total_entropy-weighted_entropy

  
  return info_gain

    


In [ ]:
information_gain(data,"toothed",target)

0.8656941534932372

##ID3

In [ ]:
def ID3(data,original_data,features,target_col=target,parent_mode_class=None):
  """
  Parameters:

    data          : It is the dataframe that keeps on changing as the tree grows. It splits accorrding to decision node depending upon the 
                    unique values in the target column

    original_data : We need to refer to the original data as and when the requirement arises in the case we need to stop the tree and so on.

    features      : Keeps the track of features that remain after a decision node is reached.
                    At the decision node the feature having highest information gain is ruled out

    target_col   : The column based on which decision has to mbe made that is branching is done based on this node

  Return Type:  This is a recursive call 

  Called In: ID3 

  About the Function:
      Using the information_gain function we would be recursively finding the column that is having the largest information 
      gain in each level of the decision tree. This is an recursive function it must have a stopping criteria which will alow
      it to reach its leaf node where probably no more decision can be taken.

      We stop the tree from growing(stopping criteria) when:
      1. All rows in the target feature have the same value(only one decision can be taken no matter what is there in the feature column)
      2. The dataset can be no longer split since there are no more features left
      3. The dataset can no longer be split since there are no more rows left / There is no data left

      By definition, we say that if the growing gets stopped because of stopping criteria two, the leaf node should predict
      the most frequently occurring target feature value of the superior (parent) node. If the growing gets stopped because of the 
      third stopping criteria, we assign the leaf node the mode target feature value of the original dataset.
  """
     ##################################################################################################################
      #1. defining the stopping criteria

      #   1a. All rows(elements) in the target column have the same value
  if len(np.unique(data[target_col]))<=1:
    return ( np.unique(data[target_col])[0])

      #  1b. The dataset can be no longer split since there are no more features left
      #        Note that:
      #        If the feature space is empty, return the mode target feature value of the direct parent node 
      #        the direct parent node is that node which has called the current run of the ID3 algorithm and hence
      #        the mode target feature value is stored in the parent_node_class variable.
  if len(features) == 0:
    return (parent_mode_class)

      #   1c.The dataset can no longer be split since there are no more rows left / There is no data left 
      #      return the mode in the target column if this is the case
  if len(data) == 0:
    return (original_data[target_col].mode()[0])

      #################################################################################################################
      # end of stopping  criterias!!!!! lets grow the treee

      # set the parent_node_class  as  the mode of the target_col
      # this variable must hold information about the mode of the target column at the current decision node 
  parent_mode_class = data[target_col].mode()[0]

      #Lets claculate the information gain for each feature and see who has the highest information_gain
  feature_info_gain =[]

  for feature in features:
       #storing the info gain of every feature present in the sub data
    feature_info_gain.append(information_gain(data,feature,target_col))

       # finding the index of the feature having highest info_gain value 
  feature_index = np.argmax(feature_info_gain)

       # best feature = feature having highest info_gain value 
  best_feature=features[feature_index]

  tree = {best_feature:{}}

       #Updating  features list by eliminating the best_feature
  features = [i for i in features if i != best_feature]

       # lets start growing the branch for different values of best_feature
  for value in np.unique(data[best_feature]):
  
    
    sub_data = data.where(data[best_feature] == value).dropna()
            
            #Call the ID3 algorithm for each of those sub_datasets with the new parameters --> Here the recursion comes in!
    subtree = ID3(sub_data,original_data,features,target_col,parent_mode_class)
             
    #Add the sub tree, grown from the sub_dataset to the tree under the root node
            #every best_feature will have value 
    tree[best_feature][value] = subtree
            
  return (tree)  
  


 

##PREDICT

In [ ]:
def predict(query,tree,default=1):
  """
  Parameters:
      query:  represents one row in a data set. This query is of dict type. 
              keys are the column name and values are the value of in the 
              column for that particular row.

      tree :  this particular parameter keeps on changing wrt the recursive calls are made. 
              This tree is prepared using the train data. As we move down the root node the 
              subsequent key values are removed starting with root node.

      default=1 helps in coming out of a situation when the key is currenty not
               in the list of keys in tree dict.



   About The Function:
               Now that we are done trainning some part of the data which resulted in growing a tree, 
               we need a function to make predictions on the test data.
               row by row we predict the target column usying the tree that is obtained.
               Procedure:
               --for every key in the query first check if the key in the query is present as the root node of the tree
                if it is the case then store result as the value of the tree's key that is root node.
                Except block will take care if the current key of the query is not present in tree's keys by retuerning default.
                After root nodes value is stored in the result then tree will have a new key that will be a interior node.
                This process keeps on repeating until leaf node is reached, to determine this situation we can check if the result is  not dictionary 
                if it is still a dictionary then we go for recursion of the function untoil leaf node is reached .
                what we actually do is we check for the particular values of the test query that is the corresponding leaf node or the decision in the tree constructed.
                suppose
                query={'breathes': 'True', 'legs': 'True', 'toothed': 'True'}
                tree =     {'legs': {'False': 'Reptile','True': {'toothed': {'False': {'breathes': {'True': 'Mammal'}},'True': 'Mammal'}}}}
                initially,
                
                list(query.keys() )   = ['breathes', 'legs', 'toothed']
                list(tree.keys()  )   = 'legs'
                when the loop reaches the index of legs for query then 
                key='legs'
                query['leg'] which is 'True'

                result = tree[key][query[key]] 
                result = tree["legs"][query["leg"]]
                       =  tree["legs"]["True"]
                         {'toothed': {'False': {'breathes': {'True': 'Mammal'}},'True': 'Mammal'}}
                so we will move towards the decision "True" for root node "legs" in tree.
               


                list(query.keys() )   = ['breathes', 'legs', 'toothed']
                list(tree.keys()  )   ='toothed'
                when the loop reaches the index of 'toothed' for query then 
                key='toothed'
                query['toothed']="True"
                result= tree["toothed"][query["toothed"]]
                        =tree["toothed"]["True"]
                        ="Mammal"

                we reached the leaf node!!!! so mammal is the final decisioon           
                """
 #1.
  for key in list(query.keys()):
      if key in list(tree.keys()):
            #2.
          try:
              result = tree[key][query[key]] 
              
          except:
              return default
  
            #3.
          result = tree[key][query[key]]
         

            #4.
          if isinstance(result,dict):

            
            return predict(query,result)

          else:
            return result


LOADING DATA INTO THE FUNCTIONS

In [ ]:
def train_test_split(dataset):
    training_data = dataset.iloc[:80].reset_index(drop=True)#We drop the index respectively relabel the index
    #starting form 0, because we do not want to run into errors regarding the row labels / indexes
    testing_data = dataset.iloc[80:].reset_index(drop=True)
    return training_data,testing_data

training_data = train_test_split(data)[0]
testing_data = train_test_split(data)[1] 



def test(data,tree):
    #Create new query instances by simply removing the target feature column from the original dataset and 
    #convert it to a dictionary
    queries = data.iloc[:,:-1].to_dict(orient = "records")
    
    #Create a empty DataFrame in whose columns the prediction of the tree are stored
    predicted = pd.DataFrame(columns=["predicted"]) 
    
    #Calculate the prediction accuracy
    for i in range(len(data)):
        predicted.loc[i,"predicted"] = predict(queries[i],tree,1.0) 
    print('The prediction accuracy is: ',(np.sum(predicted["predicted"] == data["class_type"])/len(data))*100,'%')
    





"""
Train the tree, Print the tree and predict the accuracy
"""
tree = ID3(training_data,training_data,training_data.columns[:-1])
print(tree)
test(testing_data,tree)

{'legs': {0: {'fins': {0.0: {'toothed': {0.0: 7.0, 1.0: 3.0}}, 1.0: {'eggs': {0.0: 1.0, 1.0: 4.0}}}}, 2: {'hair': {0.0: 2.0, 1.0: 1.0}}, 4: {'hair': {0.0: {'toothed': {0.0: 7.0, 1.0: 5.0}}, 1.0: 1.0}}, 6: {'aquatic': {0.0: 6.0, 1.0: 7.0}}, 8: 7.0}}
The prediction accuracy is:  85.71428571428571 %
